In [3]:
from datasets import load_dataset, load_from_disk

ds = load_from_disk("./dataset/")
#ds = load_dataset("AnonymousUserCLIP4MC/CLIP4MC")

c:\D\miniconda\envs\clip4mc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
ds.save_to_disk("./dataset/")

In [2]:
ds['train'][0]

{'vid': '-1ZE5GOu9_Y',
 'size': [0.41875,
  0.4375,
  0.43125,
  0.4625,
  0.45,
  0.45625000000000004,
  0.45625000000000004,
  0.4625,
  0.4625,
  0.4625,
  0.4625,
  0.4625,
  0.4625,
  0.4625,
  0.4625,
  0.4625],
 'transcript clip': 'i do not even know if the people who baked the cake leant to the minecraft cake or if this minecraft cake maybe came out of this picture .',
 'begin position': 64.8,
 'end position': 68.0}

In [3]:
import pickle
import yt_dlp
import cv2
import os

video_url = "https://www.youtube.com/watch?v=TIRzniTsh8Q"
output_path = "./dataset/data_dir_0/video_input.pkl" 


# Download video file
ydl_opts = {
    #'format': 'bestvideo',
    "download_sections": { "*": [(10, 30)] },
    'outtmpl': 'temp_video.mp4',  # Save video temporarily
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])
"""
# Read video and extract frames
frames = []
cap = cv2.VideoCapture("temp_video.mp4")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)
cap.release()

# Save frames as pickle
with open(output_path, "wb") as f:
    pickle.dump(frames, f)

# Cleanup temporary video
os.remove("temp_video.mp4")

print(f"Extracted {len(frames)} frames and saved in 'video_frames.pkl'")
"""


[youtube] Extracting URL: https://www.youtube.com/watch?v=TIRzniTsh8Q
[youtube] TIRzniTsh8Q: Downloading webpage
[youtube] TIRzniTsh8Q: Downloading tv client config
[youtube] TIRzniTsh8Q: Downloading player 6b3caec8
[youtube] TIRzniTsh8Q: Downloading tv player API JSON
[youtube] TIRzniTsh8Q: Downloading ios player API JSON
[youtube] TIRzniTsh8Q: Downloading m3u8 information
[info] TIRzniTsh8Q: Downloading 1 format(s): 137+140
[download] Destination: temp_video.f137.mp4
[download]  11.5% of  576.74MiB at    5.36MiB/s ETA 01:35  

KeyboardInterrupt: 

In [4]:
import numpy as np
import yt_dlp
import ffmpeg
import pickle
import os
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

def video_to_numpy(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Resize frame to (160, 256)
        frame = cv2.resize(frame, (256, 160), interpolation=cv2.INTER_AREA)

        # Convert frame to (C, H, W) -> (3, 160, 256)
        frame = np.transpose(frame, (2, 0, 1))
        
        frames.append(frame)
    
    cap.release()
    return np.array(frames)

def download_video(video_url: str, output_file: str):
    """Downloads the best video and audio and merges them into a single file."""
    ydl_opts = {
        "format": "bestvideo",
        "outtmpl": output_file,
        "merge_output_format": "mp4",  # Ensure merged MP4 output
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

def clip_video(input_file: str, start_time: int, end_time: int, output_file: str):
    """Clips the video using FFmpeg."""
    ffmpeg.input(input_file, ss=start_time, to=end_time).output(output_file, vcodec="libx264", acodec="aac").run()

def save_as_pickle(video_path: str, ti_data: list, size: list, pickle_path: str, pickle_output_ti: str, pickle_output_size: str):
    """Reads video data and saves it as a pickle file."""
    #with open(video_path, "rb") as f:
        #video_data = f.read()
    video_data = video_to_numpy(video_path)
    with open(pickle_path, "wb") as f:
        
        pickle.dump(video_data, f)

    with open(pickle_output_ti, "wb") as f:
        pickle.dump(ti_data, f)

    with open(pickle_output_size, "wb") as f:
        pickle.dump(size, f)

"""
# Define paths
video_url = "https://www.youtube.com/watch?v=TIRzniTsh8Q"
downloaded_video = "full_video.mp4"
clipped_video = "clipped_video.mp4"
pickle_output = "./dataset/data_dir_0/video_input.pkl"

# Ensure output directory exists
os.makedirs(os.path.dirname(pickle_output), exist_ok=True)

# Step 1: Download video
download_video(video_url, downloaded_video)

# Step 2: Clip between 10s-30s
clip_video(downloaded_video, 10, 30, clipped_video)

# Step 3: Save clipped video as pickle
save_as_pickle(clipped_video, pickle_output)

# Step 4: Delete videos.
os.remove(downloaded_video)
os.remove(clipped_video)

print(f"Video clipped and saved as pickle: {pickle_output}")
"""


'\n# Define paths\nvideo_url = "https://www.youtube.com/watch?v=TIRzniTsh8Q"\ndownloaded_video = "full_video.mp4"\nclipped_video = "clipped_video.mp4"\npickle_output = "./dataset/data_dir_0/video_input.pkl"\n\n# Ensure output directory exists\nos.makedirs(os.path.dirname(pickle_output), exist_ok=True)\n\n# Step 1: Download video\ndownload_video(video_url, downloaded_video)\n\n# Step 2: Clip between 10s-30s\nclip_video(downloaded_video, 10, 30, clipped_video)\n\n# Step 3: Save clipped video as pickle\nsave_as_pickle(clipped_video, pickle_output)\n\n# Step 4: Delete videos.\nos.remove(downloaded_video)\nos.remove(clipped_video)\n\nprint(f"Video clipped and saved as pickle: {pickle_output}")\n'

In [6]:
for i in range(1, 5):
    dict_ = ds['train'][i]

    # Define paths
    video_url = f"https://www.youtube.com/watch?v={dict_['vid']}"
    downloaded_video = "full_video.mp4"
    clipped_video = "clipped_video.mp4"
    pickle_output_vi = f"./dataset/data_dir_{i}/video_input.pkl"
    pickle_output_ti = f"./dataset/data_dir_{i}/text_input.pkl"
    pickle_output_size = f"./dataset/data_dir_{i}/size.json"

    #text = {"tokens": tokenizer.tokenize(dict_['transcript clip'])}
    encoded_data = tokenizer(dict_['transcript clip'], truncation=True, return_tensors="pt", max_length=tokenizer.model_max_length,  padding="max_length")
    text = {"tokens": encoded_data["input_ids"].squeeze(0)}
    size = dict_['size']

    # Ensure output directory exists
    os.makedirs(os.path.dirname(pickle_output_vi), exist_ok=True)

    # Step 1: Download video
    download_video(video_url, downloaded_video)

    # Step 2: Clip between 10s-30s
    clip_video(downloaded_video, dict_['begin position'], dict_['end position'], clipped_video)

    # Step 3: Save clipped video as pickle
    save_as_pickle(clipped_video, text, size, pickle_output_vi, pickle_output_ti, pickle_output_size)

    # Step 4: Delete videos.
    os.remove(downloaded_video)
    os.remove(clipped_video)

    print(f"Video clipped and saved as pickle")

[youtube] Extracting URL: https://www.youtube.com/watch?v=fR5ZlT95CiM
[youtube] fR5ZlT95CiM: Downloading webpage
[youtube] fR5ZlT95CiM: Downloading tv client config
[youtube] fR5ZlT95CiM: Downloading player 6b3caec8
[youtube] fR5ZlT95CiM: Downloading tv player API JSON
[youtube] fR5ZlT95CiM: Downloading ios player API JSON
[youtube] fR5ZlT95CiM: Downloading m3u8 information
[info] fR5ZlT95CiM: Downloading 1 format(s): 299
[download] Destination: full_video.mp4
[download] 100% of  674.57MiB in 00:02:25 at 4.65MiB/s      
Video clipped and saved as pickle
[youtube] Extracting URL: https://www.youtube.com/watch?v=NJwnLdAfl3o
[youtube] NJwnLdAfl3o: Downloading webpage
[youtube] NJwnLdAfl3o: Downloading tv client config
[youtube] NJwnLdAfl3o: Downloading player 6b3caec8
[youtube] NJwnLdAfl3o: Downloading tv player API JSON
[youtube] NJwnLdAfl3o: Downloading ios player API JSON
[youtube] NJwnLdAfl3o: Downloading m3u8 information
[info] NJwnLdAfl3o: Downloading 1 format(s): 137
[download] Des

In [25]:
tokens = tokenizer("Hi, How are you?", return_tensors="pt", max_length=77, truncation=True, padding="max_length") 

In [28]:
tokens["input_ids"].shape

torch.Size([1, 77])

In [10]:
import os
import pickle

def process_text_file(text_pkl_path, text_data):
    """Load text_input.pkl, tokenize text, and save it back."""
    # Load text_input.pkl
    #with open(text_pkl_path, "rb") as f:
        #text_data = pickle.load(f)['tokens']  # Expecting list of strings
    
    # Ensure data is a list of strings
    #if not isinstance(text_data, list) or not all(isinstance(t, str) for t in text_data):
    #    raise ValueError(f"Unexpected data format in {text_pkl_path}")
    
    # Tokenize text
    encoded_data = tokenizer(text_data, truncation=True, return_tensors="pt", max_length=tokenizer.model_max_length,  padding="max_length")
    
    # Save back to pickle
    with open(text_pkl_path, "wb") as f:
        pickle.dump(encoded_data["input_ids"].squeeze(0), f)

def process_video_file(video_pkl_path):
    with open(video_pkl_path, "rb") as f:
        video_data = pickle.load(f)  # Expecting list of strings
    print(video_data.shape)
    frames = []
    for frame in video_data:
        # Resize frame to (160, 256)
        #frame = cv2.resize(frame, (256, 160), interpolation=cv2.INTER_AREA)

        # Convert frame to (C, H, W) -> (3, 160, 256)
        #frame = np.transpose(frame, (2, 0, 1))
        frames.append(frame)
    print(np.array(frames).shape)
    ff
    with open(video_pkl_path, "wb") as f:
        pickle.dump(np.array(frames), f)

    

def process_all_datasets(dataset_dir):
    """Iterate over data_dir_0 to data_dir_4 and process text_input.pkl."""
    for i in range(2,5):  # data_dir_0 to data_dir_4
        dict_ = ds['train'][i]
        text = dict_['transcript clip']
        sub_dir_path = os.path.join(dataset_dir, f"data_dir_{i}")
        
        if os.path.isdir(sub_dir_path):
            text_pkl_path = os.path.join(sub_dir_path, "text_input.pkl")
            video_pkl_path = os.path.join(sub_dir_path, "video_input.pkl")
            
            if os.path.exists(text_pkl_path):
                print(f"Processing: {text_pkl_path}")
                #process_text_file(text_pkl_path, text)
                process_video_file(video_pkl_path)
            else:
                print(f"Warning: {text_pkl_path} not found.")

# Define dataset directory
dataset_path = "./dataset"
process_all_datasets(dataset_path)


Processing: ./dataset\data_dir_2\text_input.pkl
(336, 3, 160, 256)
(336, 3, 160, 256)


NameError: name 'ff' is not defined

In [1]:
import cv2
import numpy as np

def video_to_numpy(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Resize frame to (160, 256)
        frame = cv2.resize(frame, (256, 160), interpolation=cv2.INTER_AREA)

        # Convert frame to (C, H, W) -> (3, 160, 256)
        #frame = np.transpose(frame, (2, 0, 1))
        
        frames.append(frame)
    
    cap.release()
    return np.array(frames)

video_path = "clipped_video.mp4"
video_data = video_to_numpy(video_path)
video_data.shape

(0,)

In [34]:
# Define paths
video_url = "https://www.youtube.com/watch?v=TIRzniTsh8Q"
downloaded_video = "full_video.mp4"
clipped_video = "clipped_video.mp4"
pickle_output = "./dataset/data_dir_0/video_input.pkl"

# Ensure output directory exists
#os.makedirs(os.path.dirname(pickle_output), exist_ok=True)

# Step 1: Download video
download_video(video_url, downloaded_video)

# Step 2: Clip between 10s-30s
clip_video(downloaded_video, 10, 30, clipped_video)

# Step 3: Save clipped video as pickle
#save_as_pickle(clipped_video, pickle_output)

# Step 4: Delete videos.
os.remove(downloaded_video)
#os.remove(clipped_video)

print(f"Video clipped and saved as pickle: {pickle_output}")

[youtube] Extracting URL: https://www.youtube.com/watch?v=TIRzniTsh8Q
[youtube] TIRzniTsh8Q: Downloading webpage
[youtube] TIRzniTsh8Q: Downloading tv client config
[youtube] TIRzniTsh8Q: Downloading player 6b3caec8
[youtube] TIRzniTsh8Q: Downloading tv player API JSON
[youtube] TIRzniTsh8Q: Downloading ios player API JSON
[youtube] TIRzniTsh8Q: Downloading m3u8 information
[info] TIRzniTsh8Q: Downloading 1 format(s): 137
[download] Destination: full_video.mp4
[download] 100% of  576.74MiB in 00:02:04 at 4.64MiB/s      
Video clipped and saved as pickle: ./dataset/data_dir_0/video_input.pkl
